# Evals are all you need

In [ ]:
%%capture
#| default_exp clean

%pip install openai
%pip install numpy
# %pip install langchain

We start with a test item

In [ ]:

import json

# Opening JSON file
f = open('../posts.json')

# returns JSON object as 
# a dictionary
posts = json.load(f)


In [ ]:
import numpy as np

postsArr = np.array(posts)

testItems = postsArr[0:9]

In [ ]:
# testItems = [{'hnuser': 'grantmuller',
# 'date': '2024-07-08T22:39:26',
# 'comment': 'We build software that improve resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Our systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and a variety of other stakeholders throughout North America.\nWe’re growing our team and are in immediate need for a Senior Software Engineer. Here are some notes on the technical requirements:\n* C# development experience using .NET Core, WebAPI, and Entity Framework\n* Experience building browser applications using Angular and Typescript\n* Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning\n* Kubernetes, Docker, Linux\nApply here: https://apply.workable.com/irth-solutions/j/33AD66BA3A/\nWe’re growing, so new positions will open up frequently. Please check back with if this one doesn’t fit your background\nreply'}]


We need to determine:
* company name
* company location
* contact info

If any information isn't present, we need to make sure that we don't hallucinate.

In [ ]:
import os
import openai

# openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = os.environ.get('OPENAI_API_KEY')



Is the posting a job posting?

In [ ]:
def getResponse(text: str):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": text}],
    )

    result = {
        "input_text": text,
        "text": response.choices[0].message.content
    }

    return result
 

In [ ]:
import json

def toJson(response, repromptNum=None):
    # try catch here. If item fails, rerun it. If it fails more than 3 times, return None
    try:
        return json.loads(response.replace('\n',''))
    except:
        if repromptNum == None:
            repromptNum = 0
        
        if repromptNum > 3:
            return '{ "errpr": "Could not receive valid json: Too many reprompts","item": '+item+'}'
        
        repromptNum += 1
        return reprompt(response, repromptNum)
    
def reprompt(item, repromptNum=None):
    newPrompt = 'Your response: \n\n'+item.text+'was not valid json. Please try again. Here is the original request:\n' + item

    response = getResponse(newPrompt)

    print('response')
    return toJson(response['text'], repromptNum=repromptNum)


In [ ]:
def extractData(item, resultItem): 
    text = "From the following job description, extract the name of the company, the job title, job requirements and how they want to be contacted: " + item['comment'] + "\n\n-----------\n\nReturn the information as a json object with the following keys: 'company', 'job_title', 'job_requirements','contact_url'. If you can't find a particular piece of information, return 'Unavailable' for the value of the key."

    result = getResponse(text)

    response = toJson(result['text'])

    # TODO: add validation to check if the response is valid
    resultItem['company'] = response['company']
    resultItem['job_title'] = response['job_title']
    resultItem['job_requirements'] = response['job_requirements']
    resultItem['contact_url'] = response['contact_url']

    print(resultItem)



In [ ]:
def isJob(item, resultItem):
    text='Is the following message a job posting, or something else? If it is a job posting, reply { "isPosting: true }. If it is not a job posting, reply with { "isPosting": false }\n-----------------\n'+ item['comment']

    result = getResponse(text)
    resp = toJson(result['text'])
 
    resultItem['isPosting'] = resp['isPosting']

    return resultItem['isPosting']

In [ ]:
def processItem(item):
    resultItem = { 'isPosting': 'whoops'}

    if isJob(item, resultItem):
        extractData(item, resultItem)

    return resultItem

In [ ]:
resultData = []

for item in testItems:
    datum = processItem(item)
    print(item)
    resultData.append(datum)

print(resultData)

In [ ]:
import json

with open('../postData.json', 'w', encoding='utf-8') as f:
    json.dump(resultData, f, ensure_ascii=False, indent=4)